In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
import timeit

In [2]:
# Importing the required notebooks
%run DifferentialEvolution.ipynb
%run IslandMethod.ipynb

In [3]:
# Funtion for which to measure time
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

In [4]:
# Start spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('IPDE') \
    .getOrCreate()
sc = spark.sparkContext
        
# Setting the test
rosenberg = lambda x: 100 * (x[1] - x[0] * x[0])**2 + (1 - x[0])**2
diffEvo = DifferentialEvolution(rosenberg, np = 20, d = 2, bound = 0.1)
island = IslandMethod(diffEvo = diffEvo, workers = 1, localIt = 100)

# Executing the island
wrapped = wrapper(island.execute, sc)
print("Execution time: ", timeit.timeit(wrapped, number = 1), "s", sep = '')


Number of Evaluations 20

Initial partition structure: [[(0, (array([-0.06580369,  0.07490479]), 1.6340159009833695)), (1, (array([ 0.01971032, -0.08837274]), 1.7488235034922774)), (2, (array([ 0.04222849, -0.01893171]), 0.96023718185746321)), (3, (array([-0.07201323,  0.08110919]), 1.725646824357836)), (4, (array([ 0.04540111,  0.03360814]), 1.0107796002078795)), (5, (array([-0.09323382,  0.08571991]), 1.7884816489640802)), (6, (array([ 0.06025012, -0.04074268]), 1.0800239497059847)), (7, (array([ 0.0193439 , -0.05659124]), 1.2861923322508451)), (8, (array([-0.05581087,  0.06324552]), 1.47630630181003)), (9, (array([-0.05847464, -0.0759558 ]), 1.7504089613821936))], [(10, (array([ 0.08806227,  0.06733476]), 1.1866056816580173)), (11, (array([-0.01103482,  0.02068644]), 1.0644820015719647)), (12, (array([ 0.05792145,  0.07538804]), 1.4063894613836014)), (13, (array([ 0.03831279, -0.0788322 ]), 1.5696523561051692)), (14, (array([ 0.00779177,  0.0892233 ]), 1.7794739549107008)), (15, (a